The iSTTC concat ACF in some units show strange behavior - the curve goes up at the last 2-3 lags, so the autocorrelation is getting high. 
Unit0 from PFdl has this (trail avg look fine).

Here check:
* load already calculated ACFs, plot on one grid, check values
* try "global" normalization - the T term is calculated based on the non shifted signal (concat trials without 0s)
* shifting the whole signal? 

In [1]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import seaborn as sns

# import from scripts
import os
current_wd = os.getcwd()
os.chdir(os.path.abspath("..\\..\\..\\isttc\\scripts"))
from calculate_acf import acf_sttc_trial_concat_global, acf_sttc_trial_concat
from cfg_global import project_folder_path
os.chdir(current_wd)

In [2]:
results_folder = 'Q:\\Personal\\Irina\\projects\\isttc\\results\\monkey\\fixation_period_1000ms_no_empty\\'
data_folder = project_folder_path + 'results\\monkey\\'

#### load already calculated ACFs, plot on one grid, check values

In [6]:
area = 'pfp'

n_lags = 20
acf_cols = ['acf_' + str(i) for i in range(n_lags)]

acf_pearsonr_trial_avg_df = pd.read_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_pearsonr_trial_avg_1000ms_no_empty_50ms_20lags_df.pkl')
acf_matrix_pearsonr_trial_avg_l = np.load(results_folder + 'binned\\' + area + '\\acf\\acf_matrix_pearsonr_trial_avg_1000ms_no_empty_50ms_20lags_df.npy')

acf_sttc_trial_avg_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_avg_1000ms_no_empty_50ms_20lags_df.pkl')
acf_matrix_sttc_trial_avg_l = np.load(results_folder + 'non_binned\\' + area + '\\acf\\acf_matrix_sttc_trial_avg_1000ms_no_empty_50ms_20lags_df.npy')

acf_sttc_trial_concat_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_concat_1000ms_no_empty_50ms_20lags_df.pkl')
acf_sttc_trial_concat_fixed_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_concat_fixed_1000ms_no_empty_50ms_20lags_df.pkl')
# acf_sttc_trial_concat_global_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_concat_global_1000ms_no_empty_50ms_20lags_df.pkl')

#### try "global" normalization - the T term is calculated based on the non shifted signal (concat trials without 0s)

In [4]:
# non-binned data
csv_data_file = data_folder + 'data_' + area + '_fixon_1000ms_no_empty_fixation.csv'
with open(csv_data_file, newline='') as f:
    reader = csv.reader(f)
    sua_non_binned_list = list(reader)
    
n_non_binned_spike_trains = len(sua_non_binned_list)
print('N spike_trains in {}: {}'.format(area, n_non_binned_spike_trains))

# transform data to a dict, key is unit_id, values is a list of spike trains (one spike train per trial)
units_dict = {}
for spike_train in sua_non_binned_list:
    spike_train_ = np.asarray(spike_train[5:]).astype(int)
    # spike_train_1000 = spike_train_[spike_train_ <= 1000]
    if int(spike_train[0]) in units_dict:
        units_dict[int(spike_train[0])].append(spike_train_)
    else:
        units_dict[int(spike_train[0])] = []
        units_dict[int(spike_train[0])].append(spike_train_)

n_non_binned_units = len(units_dict)
print('n units {}'.format(n_non_binned_units))

N spike_trains in pfp: 24668
n units 543


In [5]:
# params for sttc
bin_size = 50
sttc_dt = 49
trial_len = n_lags * bin_size

acf_sttc_trial_concat_l = []
unit_id_calc_l = []

for k,v in units_dict.items():
    print('Processing unit {}, n trials {}'.format(k, len(v)))
    
    if len(v) <= 1:
        print('ONLY 1 TRIAL: can not calculate, skipping...')
    else:
        acf_concat = acf_sttc_trial_concat(v, n_lags_=n_lags, lag_shift_=bin_size, sttc_dt_=sttc_dt, trial_len_=trial_len,
                                           zero_padding_len_=3000, verbose_=False)
        acf_sttc_trial_concat_l.append(acf_concat)
        unit_id_calc_l.append(k)

acf_sttc_trial_concat_global_df = pd.DataFrame(np.array(acf_sttc_trial_concat_l), columns=acf_cols)
acf_sttc_trial_concat_global_df.insert(0, 'unit_id', unit_id_calc_l)

print('NaNs in acf {}'.format(acf_sttc_trial_concat_global_df.isnull().any().any()))
acf_sttc_trial_concat_global_df.head(3)

acf_sttc_trial_concat_global_df.to_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_concat_fixed_1000ms_no_empty_50ms_20lags_df.pkl')

Processing unit 0, n trials 77
Processing unit 1, n trials 15
Processing unit 2, n trials 25
Processing unit 3, n trials 27
Processing unit 4, n trials 18
Processing unit 5, n trials 25
Processing unit 6, n trials 18
Processing unit 7, n trials 42
Processing unit 8, n trials 46
Processing unit 9, n trials 31
Processing unit 10, n trials 15
Processing unit 11, n trials 21
Processing unit 12, n trials 12
Processing unit 13, n trials 77
Processing unit 14, n trials 54
Processing unit 15, n trials 81
Processing unit 16, n trials 107
Processing unit 17, n trials 49
Processing unit 18, n trials 39
Processing unit 19, n trials 10
Processing unit 20, n trials 23
Processing unit 21, n trials 13
Processing unit 22, n trials 57
Processing unit 23, n trials 116
Processing unit 24, n trials 31
Processing unit 25, n trials 71
Processing unit 26, n trials 98
Processing unit 27, n trials 15
Processing unit 28, n trials 52
Processing unit 29, n trials 32
Processing unit 30, n trials 130
Processing unit

In [7]:
fig_folder = results_folder + '\\acf_line_plots\\pfp_one_plot_per_unit_wtf\\'

for unit_id in acf_pearsonr_trial_avg_df['unit_id'].values:

    fig, axes = plt.subplots(1,1, figsize=(5,4))
    
    # unit_id = 33
    
    axes.plot(acf_pearsonr_trial_avg_df.query('unit_id == @unit_id')[acf_cols].values[0], c='slategray', lw=0.5 ,marker='o', markersize='4', 
              label='personr_trial_avg')
    axes.plot(acf_sttc_trial_avg_df.query('unit_id == @unit_id')[acf_cols].values[0], c='steelblue', lw=0.5 ,marker='o', markersize='4', 
              label='sttc_trial_avg')
    axes.plot(acf_sttc_trial_concat_df.query('unit_id == @unit_id')[acf_cols].values[0], c='salmon', lw=0.5 ,marker='o', markersize='4', 
              label='sttc_trial_concat')
    axes.plot(acf_sttc_trial_concat_global_df.query('unit_id == @unit_id')[acf_cols].values[0], c='firebrick', lw=0.5 ,marker='o', markersize='4', 
              label='sttc_trial_concat_global')
    axes.plot(acf_sttc_trial_concat_fixed_df.query('unit_id == @unit_id')[acf_cols].values[0], c='teal', lw=0.5 ,marker='o', markersize='4', 
          label='sttc_trial_concat_wtf')

    axes.axhline(y=0, lw=0.5, c='k')
    
    axes.legend(frameon=False)
    
    fig.suptitle('acf, unit ' + str(unit_id))
    
    axes.set_ylabel('ACF (a.u.)')
    axes.set_xlabel('Time (ms)')
    axes.set_xticks(np.linspace(0, 20, 11).astype(int))
    axes.set_xticklabels(np.linspace(0, 1000, 11).astype(int))
    axes.set_xlim([0,20])
    
    sns.despine()
    
    fig.savefig(fig_folder + 'acf_1000ms_no_empty_50ms_20lags_unit_' + str(unit_id) + '.png' , bbox_inches='tight')
    plt.close()

#### Calculate isttc concat for one unit

In [ ]:
unit = units_dict[0]
unit[:10]

In [ ]:
bin_size = 50
sttc_dt = 49
trial_len = n_lags * bin_size

unit = units_dict[0]
acf_concat = acf_sttc_trial_concat(unit, n_lags_=n_lags, lag_shift_=bin_size, sttc_dt_=sttc_dt, trial_len_=trial_len,
                                   zero_padding_len_=3000, verbose_=True)

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(5,4))
    
unit_id = 0

axes.plot(acf_pearsonr_trial_avg_df.query('unit_id == @unit_id')[acf_cols].values[0], c='slategray', lw=0.5 ,marker='o', markersize='4', 
          label='personr_trial_avg')
axes.plot(acf_sttc_trial_avg_df.query('unit_id == @unit_id')[acf_cols].values[0], c='steelblue', lw=0.5 ,marker='o', markersize='4', 
          label='sttc_trial_avg')
axes.plot(acf_sttc_trial_concat_df.query('unit_id == @unit_id')[acf_cols].values[0], c='salmon', lw=0.5 ,marker='o', markersize='4', 
          label='sttc_trial_concat')
axes.plot(acf_sttc_trial_concat_global_df.query('unit_id == @unit_id')[acf_cols].values[0], c='firebrick', lw=0.5 ,marker='o', markersize='4', 
          label='sttc_trial_concat_global')
axes.plot(acf_concat, c='teal', lw=0.5 ,marker='o', markersize='4', 
          label='sttc_trial_concat_wtf')

axes.axhline(y=0, lw=0.5, c='k')

axes.legend(frameon=False)

fig.suptitle('acf, unit ' + str(unit_id))

axes.set_ylabel('ACF (a.u.)')
axes.set_xlabel('Time (ms)')
axes.set_xticks(np.linspace(0, 20, 11).astype(int))
axes.set_xticklabels(np.linspace(0, 1000, 11).astype(int))
axes.set_xlim([0,20])

sns.despine()

In [ ]:
acf_concat_global = acf_sttc_trial_concat_global(unit[:10], n_lags_=n_lags, lag_shift_=bin_size, sttc_dt_=sttc_dt, trial_len_=trial_len,
                                   zero_padding_len_=3000, verbose_=True)